In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k
from src.baseline_models import random_recommendation, popularity_recommendation

In [2]:
data = pd.read_csv('../raw_data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

In [5]:
%%time

# Создадим датафрейм с item_id и весом товара
items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
# Уберем из списка товары, сумма продаж которых меньше 1. Иначе получаются отрицательные значения в log.
# Может подразумевалось иначе, но я решил сделать так, т.к. рекомендовать непопулярные товары смысла нет.
items_weights = items_weights[items_weights['sales_value'] >= 1]
# Посчитаем веса
items_weights['weight'] = np.log(items_weights['sales_value'] + 0.01)
# Отнормируем веса
items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()
items_weights.drop('sales_value', axis=1, inplace=True)

# Выполним функцию
weighted_random_recommendation(items_weights, n=5)

Wall time: 640 ms


[1045214, 5567393, 12577269, 15924982, 5725201]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [6]:
result = pd.read_csv('../predictions/predictions_basic.csv',)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


Преобразуем значения из строк в список из чисел

In [7]:
for alg_col in result.iloc[:,1:]:
    result[alg_col] = result.apply(lambda row: [int(x.rstrip(',')) for x in row[alg_col].strip('][').split()], axis=1)

In [8]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


Добавим результаты алгоритма weighted_random_recommendation

In [9]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

In [10]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[13672258, 9883986, 12487324, 13158502, 988030]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[871645, 12330439, 13133947, 1007746, 936924]"


Рассчитаем метрику качества для каждого алгоритма

In [11]:
for alg_col in result.iloc[:,2:-1]:
    result['user_id'].apply(lambda x: precision_at_k(result[alg_col], result['actual'], k=5))

In [12]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual'], k=5), axis=1).mean()

0.0005876591576885406

In [13]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'], k=5), axis=1).mean()

0.15523996082272082

In [14]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean()

0.033594515181195064

In [15]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], k=5), axis=1).mean()

0.03525954946131262

In [16]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean()

0.03614103819784544

In [17]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual'], k=5), axis=1).mean()

0.1799869409076044

In [18]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual'], k=5), axis=1).mean()

0.0008814887365328109

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров

In [19]:
# Составим таблицу популярных товаром по количеству продаж
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [20]:
# Выделим 5000 самых популярных
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)
top_5000.head()

,item_id,n_sold
55470,6534178,190227964
55430,6533889,15978434
55465,6534166,12439291
55576,6544236,2501949
43620,1404121,1562004


In [21]:
# Сделаем рекомендацию
items = top_5000.item_id.unique()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[13672258, 9883986, 12487324, 13158502, 988030]","[967065, 1119060, 957190, 1130777, 879143]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[871645, 12330439, 13133947, 1007746, 936924]","[953155, 1135694, 940108, 853235, 935302]"


In [22]:
result.apply(lambda row: precision_at_k(row['random_recommendation_5000'], row['actual'], k=5), axis=1).mean()

0.005680705190989223

Видим, что алгоритм random_recommendation на топ 5000 товаров стал работать лучше чем на всех товарах.
0.00441 против 0.00059

Алгоритм popularity_recommendation будет давать такие же результаты на топ 5000 как и на всем наборе, т.к. нет разницы рекомендовать топ 5 из из всех или из топ 5000. С оговоркой, что у нас популярность идет по суммарной цене, которую принес товар.

In [23]:
# Пересчитаем веса для топ 5000 товаров
items_weights_5000 = top_5000
items_weights_5000['weight'] = np.log(items_weights_5000['n_sold'])
items_weights_5000['weight'] = items_weights_5000['weight'] / items_weights_5000['weight'].sum()
items_weights_5000.drop('n_sold', axis=1, inplace=True)
items_weights_5000.head()

,item_id,weight
55470,6534178,0.000680
55430,6533889,0.000592
55465,6534166,0.000583
55576,6544236,0.000525
43620,1404121,0.000509


In [24]:
items_weights_5000['weight'].sum()

1.0000000000000002

In [25]:
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights_5000, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[13672258, 9883986, 12487324, 13158502, 988030]","[967065, 1119060, 957190, 1130777, 879143]","[15926775, 1070820, 1051001, 1030093, 1034686]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[871645, 12330439, 13133947, 1007746, 936924]","[953155, 1135694, 940108, 853235, 935302]","[954651, 887661, 946977, 1074697, 962522]"


In [26]:
result.apply(lambda row: precision_at_k(row['weighted_random_recommendation_5000'], row['actual'], k=5), axis=1).mean()

0.0073457394711067495

Видим, что алгоритм weighted_random_recommendation на топ 5000 товаров стал работать лучше чем на всех товарах. 0.00754 против 0.00147

- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [27]:
data_train

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2282320,222,41297772783,635,1120741,1,0.59,304,0.00,1716,91,0.0,0.0
2282321,462,41297773713,635,993339,1,1.99,304,0.00,2040,91,0.0,0.0
2282322,462,41297773713,635,995242,1,1.00,304,-0.89,2040,91,0.0,0.0
2282323,462,41297773713,635,10180324,1,3.00,304,-0.29,2040,91,0.0,0.0


In [28]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000.item_id), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

C:\Users\Spoukster\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Создадим отдельную таблицу результатов, для оценки лушего количества соседей для алгоритмов ItemItem

In [30]:
result_itemitem = result[['user_id', 'actual']]
result_itemitem.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


ItemItemRecommender

In [31]:
%%time

precision = {}
for i in range(2, 8):
    model = ItemItemRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result_itemitem[f'itemitem_K{i}'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    # Криво, но просто
    precision[result_itemitem.\
                     apply(lambda row: precision_at_k(row[f'itemitem_K{i}'], row['actual'], k=5), axis=1).mean()] = i

C:\Users\Spoukster\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':



Wall time: 7.2 s


In [32]:
f'Лучшее качество {max(precision)} получается при {precision[max(precision)]} соседях'

'Лучшее качество 0.059786157362063795 получается при 2 соседях'

In [33]:
# Для сравнения при K=5
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], k=5), axis=1).mean()

0.033594515181195064

CosineRecommender

In [34]:
%%time

precision = {}
for i in range(2, 8):
    model = CosineRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result_itemitem[f'cosine_K{i}'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    # Криво, но просто
    precision[result_itemitem.\
                     apply(lambda row: precision_at_k(row[f'cosine_K{i}'], row['actual'], k=5), axis=1).mean()] = i

C:\Users\Spoukster\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':



Wall time: 7.55 s


In [35]:
f'Лучшее качество {max(precision)} получается при {precision[max(precision)]} соседях'

'Лучшее качество 0.03672869735553398 получается при 6 соседях'

In [36]:
# Для сравнения при K=5
result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], k=5), axis=1).mean()

0.03525954946131262

TFIDFRecommender

In [37]:
%%time

precision = {}
for i in range(2, 8):
    model = TFIDFRecommender(K=i, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result_itemitem[f'tfidf_K{i}'] = result_itemitem['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])
    # Криво, но просто
    precision[result_itemitem.\
                     apply(lambda row: precision_at_k(row[f'tfidf_K{i}'], row['actual'], k=5), axis=1).mean()] = i


Wall time: 7.73 s


In [38]:
f'Лучшее качество {max(precision)} получается при {precision[max(precision)]} соседях'

'Лучшее качество 0.03741429970617061 получается при 6 соседях'

In [39]:
# Для сравнения при K=5
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], k=5), axis=1).mean()

0.03614103819784544